### Image Similarlity for NFTs Collection by extracting image features and cosine similarity to get the top N nearest neighbors

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from keras.applications.xception import Xception, preprocess_input
import tensorflow as tf
from keras.preprocessing import image
from keras.layers import Input
from keras.backend import reshape
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

2022-05-01 18:51:02.447447: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/c/Users/pavan/ACAD/LBU/ISR/catkin_ws/devel/lib:/opt/ros/noetic/lib
2022-05-01 18:51:02.447507: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
witches_images_dir = 'witches_data/witch_images'
print(witches_images_dir)

witches_data/witch_images


In [3]:
def get_image_paths(path):
    # Combine image paths into a single list
    image_names = []
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            if 'os.png' in filename: 
                fullpath = os.path.join(dirname, filename)
                image_names.append(fullpath)
    return image_names


def preprocess_img(img_path):
    dsize = (225, 225)
    pre_image = cv2.imread(img_path)
    pre_image = cv2.resize(pre_image, dsize, interpolation=cv2.INTER_NEAREST)  
    pre_image = np.expand_dims(pre_image, axis=0)
    pre_image = preprocess_input(pre_image)
    return pre_image


def model():
    model = Xception(weights='imagenet', include_top=False)
    for layer in model.layers:
        layer.trainable=False
        model.summary()
    return model


def feature_extraction(image_data, model):
    features = model.predict(image_data)
    features = np.array(features)
    features = features.flatten()
    return features


def result_vector_cosine(model, feature_vector, new_img, n_results):
    new_feature = model.predict(new_img)
    new_feature = np.array(new_feature)
    new_feature = new_feature.flatten()
    nbrs = NearestNeighbors(n_neighbors=n_results, metric="cosine").fit(feature_vector)
    distances, indices = nbrs.kneighbors([new_feature])
    return (indices)


def show_query_image(data):
    plt.title("Query Image and Images most similar to queried image")
    plt.imshow(data)
    plt.xticks([])
    plt.yticks([])


# size set t0 12
def show_result(data, result):
    fig = plt.figure(figsize=(12,8))
    for i in range(0,12):
        index_result = result[0][i]
        plt.subplot(3, 4, i+1)
        plt.imshow(cv2.imread(data[index_result]))
        plt.xticks([])
        plt.yticks([])
    plt.show()

In [4]:
images = get_image_paths(witches_images_dir)

In [5]:
print(len(images))
images[:5]

9761


['witches_data/witch_images/1/1_os.png',
 'witches_data/witch_images/10/10_os.png',
 'witches_data/witch_images/100/100_os.png',
 'witches_data/witch_images/1000/1000_os.png',
 'witches_data/witch_images/1001/1001_os.png']

In [6]:
%%time
# extract features for each image
features = []
main_model = model()

for i in images[1000:2000]:  # only extract the first 1000 to save time, remove [:1000] as you see fit
    new_img = preprocess_img(i)
    features.append(feature_extraction(new_img, main_model))

feature_vec = np.array(features)


2022-05-01 18:56:38.351718: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-01 18:56:38.356987: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/c/Users/pavan/ACAD/LBU/ISR/catkin_ws/devel/lib:/opt/ros/noetic/lib
2022-05-01 18:56:38.359353: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/c/Users/pavan/ACAD/LBU/ISR/catkin_ws/devel/lib:/opt/ros/noetic/lib
2022-05-01 18:56:38.361327: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.

In [1]:
# change the number to try different query image

query_image = images[946]

# print query image name
print(query_image)

# result are the index numbers for the 12 most similar images
top_similar_image_indexes = result_vector_cosine(main_model, feature_vec, preprocess_img(query_image), 16)
print(top_similar_image_indexes)
# print result image names
for i in range(0,16):
    j = top_similar_image_indexes[0][i]
    print(images[j])

show_query_image(cv2.imread(query_image))

show_result(images, top_similar_image_indexes)

NameError: name 'images' is not defined